In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
from pandas import DataFrame

In [3]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [4]:
def url(i):
    # Permet d'obtenir l'url des 
    if i == 0:
        url_0 = 'http://fpifrance.fr/fr/annuaire-adherents?f%5B0%5D=im_field_chambre_regional%3A45&f%5B1%5D=im_field_chambre_regional%3A46&f%5B2%5D=im_field_chambre_regional%3A39&f%5B3%5D=im_field_chambre_regional%3A44&f%5B4%5D=im_field_chambre_regional%3A27&f%5B5%5D=im_field_chambre_regional%3A42&f%5B6%5D=im_field_chambre_regional%3A26&f%5B7%5D=im_field_chambre_regional%3A33&f%5B8%5D=im_field_chambre_regional%3A34&f%5B9%5D=im_field_chambre_regional%3A32&f%5B10%5D=im_field_chambre_regional%3A31&f%5B11%5D=im_field_chambre_regional%3A30&f%5B12%5D=im_field_chambre_regional%3A36&f%5B13%5D=im_field_chambre_regional%3A37&f%5B14%5D=im_field_chambre_regional%3A40&f%5B15%5D=im_field_chambre_regional%3A38&f%5B16%5D=im_field_chambre_regional%3A43&f%5B17%5D=im_field_chambre_regional%3A29&f%5B18%5D=im_field_chambre_regional%3A73'
        return url_0
    else:
        u = 'http://fpifrance.fr/fr/annuaire-adherents?page='
        r = str(i-1)
        l = '&f%5B0%5D=im_field_chambre_regional%3A45&f%5B1%5D=im_field_chambre_regional%3A46&f%5B2%5D=im_field_chambre_regional%3A39&f%5B3%5D=im_field_chambre_regional%3A44&f%5B4%5D=im_field_chambre_regional%3A27&f%5B5%5D=im_field_chambre_regional%3A42&f%5B6%5D=im_field_chambre_regional%3A26&f%5B7%5D=im_field_chambre_regional%3A33&f%5B8%5D=im_field_chambre_regional%3A34&f%5B9%5D=im_field_chambre_regional%3A32&f%5B10%5D=im_field_chambre_regional%3A31&f%5B11%5D=im_field_chambre_regional%3A30&f%5B12%5D=im_field_chambre_regional%3A36&f%5B13%5D=im_field_chambre_regional%3A37&f%5B14%5D=im_field_chambre_regional%3A40&f%5B15%5D=im_field_chambre_regional%3A38&f%5B16%5D=im_field_chambre_regional%3A43&f%5B17%5D=im_field_chambre_regional%3A29&f%5B18%5D=im_field_chambre_regional%3A73'
        return u+r+l


In [5]:
df = DataFrame(columns = ['Promoteur','Adresse','Tel','Mail','Site','Structure','Capital','Siret'])
for i in range(1000):
    df.loc[i]=[float('nan'),float('nan'),float('nan'),float('nan'),float('nan'),float('nan'),float('nan'),float('nan')]

In [6]:
def get_data(i,start_row):

    response = simple_get(url(i))
    compteur = start_row

    if response is not None:
        
        html = BeautifulSoup(response, 'html.parser')
        allDivs = html.find_all('div', attrs={'class': 'node node-adherent node-promoted annuaire_adherent clearfix'})
        
        for div in allDivs:
            
            new_url = div.find('a').get('href')
            res = simple_get('http://fpifrance.fr'+ new_url)
            html2 = BeautifulSoup(res, 'html.parser')
            u_div = html2.find('div', attrs = {'class':'content'})
            
            # Nom du promoteur
            name_box = div.find('h2')
            df.loc[compteur]['Promoteur'] = name_box.text
            
            # Adresse du promoteur
            under_div = div.find('div', attrs = {'class':'field field-name-field-adresse field-type-text-long field-label-above'})
            if under_div is not None:
                post_box = under_div.find('div', attrs = {'class':'field-item even'})
                df.loc[compteur]['Adresse'] = zip_code(post_box.text)
                
            # Numéro de téléphone
            under_div = div.find('div', attrs = {'class':'field field-name-field-telephone field-type-text field-label-above'})
            if under_div is not None:
                tel_box = under_div.find('div', attrs = {'class':'field-item even'})
                df.loc[compteur]['Tel'] = tel_box.text
                
            # E-mail
            under_div = u_div.find('div', attrs = {'class':'field field-name-field-email field-type-text field-label-above'})
            if under_div is not None:
                box = under_div.find('a')
                df.loc[compteur]['Mail'] = box.text
                
            # Site internet
            under_div = u_div.find('div', attrs = {'class':'field field-name-field-url field-type-link-field field-label-above'})
            if under_div is not None:
                box = under_div.find('div', attrs = {'class':'field-item even'})
                df.loc[compteur]['Site'] = box.text
                
            # Forme juridique
            under_div = u_div.find('div', attrs = {'class':'field field-name-field-forme-juridique field-type-text field-label-above'})
            if under_div is not None:
                box = under_div.find('div', attrs = {'class':'field-item even'})
                df.loc[compteur]['Structure'] = box.text
                
            # Capital social
            under_div = u_div.find('div', attrs = {'class':'field field-name-field-capital-social field-type-text field-label-above'})
            if under_div is not None:
                box = under_div.find('div', attrs = {'class':'field-item even'})
                df.loc[compteur]['Capital'] = box.text
                
            # Siret
            under_div = u_div.find('div', attrs = {'class':'field field-name-field-no-siret field-type-text field-label-above'})
            if under_div is not None:
                box = under_div.find('div', attrs = {'class':'field-item even'})
                df.loc[compteur]['Siret'] = box.text
                
            compteur += 1
                            
        return compteur 

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))

In [7]:
N = 57

def get_all_data():
    s_init = get_data(0,0)
    for i in range(1,N):
        s_next = get_data(i,s_init)
        s_init = s_next
        print(i)   

In [12]:
url(3)

'http://fpifrance.fr/fr/annuaire-adherents?page=2&f%5B0%5D=im_field_chambre_regional%3A45&f%5B1%5D=im_field_chambre_regional%3A46&f%5B2%5D=im_field_chambre_regional%3A39&f%5B3%5D=im_field_chambre_regional%3A44&f%5B4%5D=im_field_chambre_regional%3A27&f%5B5%5D=im_field_chambre_regional%3A42&f%5B6%5D=im_field_chambre_regional%3A26&f%5B7%5D=im_field_chambre_regional%3A33&f%5B8%5D=im_field_chambre_regional%3A34&f%5B9%5D=im_field_chambre_regional%3A32&f%5B10%5D=im_field_chambre_regional%3A31&f%5B11%5D=im_field_chambre_regional%3A30&f%5B12%5D=im_field_chambre_regional%3A36&f%5B13%5D=im_field_chambre_regional%3A37&f%5B14%5D=im_field_chambre_regional%3A40&f%5B15%5D=im_field_chambre_regional%3A38&f%5B16%5D=im_field_chambre_regional%3A43&f%5B17%5D=im_field_chambre_regional%3A29&f%5B18%5D=im_field_chambre_regional%3A73'

In [9]:
df

,Promoteur,Adresse,Tel,Mail,Site,Structure,Capital,Siret
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_excel('/Users/stephaneblondellarougery/Desktop/promoteurs2.xlsx', sheet_name='sheet1', index=False)

In [1]:
texte = '3, rue Peyrolieres - 31000 Toulouse'

alphabet = []
for letter in range(65, 123):
        alphabet.append(chr(letter))
        
def zip_code(texte):
    L = []
    for x in texte.split(" - "):
        L.append(x)
        
    retour = L[len(L)-1]
    
    M = []
    for x in retour.split(" "):
        M.append(x)
    
    return int(M[0])

zip_code(texte)

31000